In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [40]:
df= pd.read_csv("./outputs/mistake_preds_4cv_650+len_lr_2024.csv.gz")

df_full = df.copy()
df.columns

Index(['Sequence', 'Length', 'virus', 'model_pred_proba_vir', 'model_pred_vir',
       'mistake', 'Baltimore', 'Family', 'Genome Composition', 'Genus',
       'Keywords', 'Mass', 'Organism', 'Protein names', 'Taxonomic lineage',
       'UR50_Cluster ID', 'Virus hosts', 'av_mw', 'genus_human_host',
       'human_host'],
      dtype='object')

In [41]:
df["virus"].value_counts()

virus
0    18418
1     6699
Name: count, dtype: int64

In [42]:
df.nunique()

Sequence                25117
Length                   1483
virus                       2
model_pred_proba_vir     1832
model_pred_vir              2
mistake                     2
Baltimore                   6
Family                     50
Genome Composition          9
Genus                     171
Keywords                15979
Mass                    22128
Organism                 1136
Protein names           21392
Taxonomic lineage         561
UR50_Cluster ID         20884
Virus hosts               291
av_mw                    2110
genus_human_host            2
human_host                  2
dtype: int64

In [43]:
df["Keywords"].drop_duplicates()

0        3D-structure . Acetylation . Phosphoprotein . ...
1                                               ANK repeat
2        Cytoplasm . Immunity . Inflammatory response ....
3        Alternative splicing . Nucleus . Reference pro...
4        3D-structure . Alternative splicing . ATP-bind...
                               ...                        
25110    3D-structure . Acetylation . Alternative splic...
25112    3D-structure . Adaptive immunity . Alternative...
25113    Alternative splicing . Coiled coil . Direct pr...
25114    Acetylation . Alternative splicing . Cell proj...
25116    Alternative splicing . Developmental protein ....
Name: Keywords, Length: 15980, dtype: object

In [44]:
df = df[["mistake","virus","Baltimore","genus_human_host"]]

In [45]:
df = pd.get_dummies(df, columns=["Baltimore","genus_human_host"],dummy_na=True)


In [46]:
y = df.pop("mistake")

In [47]:
df

,virus,Baltimore_dsDNA,Baltimore_dsDNA-RT,Baltimore_dsRNA,Baltimore_ssDNA,Baltimore_ssRNA,Baltimore_ssRNA-RT,Baltimore_nan,genus_human_host_False,genus_human_host_True,genus_human_host_nan
0,0,False,False,False,False,False,False,True,False,False,True
1,1,True,False,False,False,False,False,False,True,False,False
2,0,False,False,False,False,False,False,True,False,False,True
3,0,False,False,False,False,False,False,True,False,False,True
4,0,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
25112,0,False,False,False,False,False,False,True,False,False,True
25113,0,False,False,False,False,False,False,True,False,False,True
25114,0,False,False,False,False,False,False,True,False,False,True
25115,0,False,False,False,False,False,False,True,False,False,True


In [53]:
# # Simplify the approach by using only numeric features for the propensity score estimation
# numeric_features_for_propensity = df.select_dtypes(include=['int64', 'float64']).columns

# Re-define the logistic regression model for propensity score estimation with numeric features only
propensity_model_simplified = LogisticRegression(max_iter=700)
propensity_model_simplified.fit(df, y)

# Recalculate propensity scores with the simplified model
propensity_scores_simplified = propensity_model_simplified.predict_proba(df)[:, 1]

# Calculate the inverse of propensity scores as weights, avoiding division by zero
weights_simplified = 1 / np.clip(propensity_scores_simplified, 0.05, 0.95)


In [54]:
weights_simplified

array([20.        , 12.06539583, 20.        , ..., 20.        ,
       20.        , 20.        ])

In [58]:
df_full["ipw_mistake_balt_vir_host"].describe()

count    25117.000000
mean        17.775423
std          4.029777
min          3.054000
25%         12.288000
50%         20.000000
75%         20.000000
max         20.000000
Name: ipw_mistake_balt_vir_host, dtype: float64

In [56]:
df_full["ipw_mistake_balt_vir_host"] = weights_simplified.round(3)
df_full.to_csv("mistakes_with_ipw.csv.gz",index=False,compression="gzip")

,Sequence,Length,virus,model_pred_proba_vir,model_pred_vir,mistake,Baltimore,Family,Genome Composition,Genus,...,Mass,Organism,Protein names,Taxonomic lineage,UR50_Cluster ID,Virus hosts,av_mw,genus_human_host,human_host,ipw_mistake_balt_vir_host
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,102,0,0.0000,0,False,NaN,NaN,NaN,NaN,...,11835,Homo sapiens (Human),DET1- and DDB1-associated protein 1 (Placenta ...,"cellular organisms, Eukaryota, Opisthokonta, M...",Q9BW61,NaN,116.03,NaN,NaN,20.000
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,362,1,1.0000,1,False,dsDNA,Asfarviridae,dsDNA,Asfivirus,...,42605,African swine fever virus (isolate Tick/South ...,Protein MGF 360-19R,"Viruses, Varidnaviria, Bamfordvirae (kingdom),...",NaN,Ornithodoros (relapsing fever ticks) [TaxID: 6...,117.69,False,False,12.065
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,97,0,0.0002,0,False,NaN,NaN,NaN,NaN,...,10795,Homo sapiens (Human),Pyrin domain-containing protein 2 (Pyrin-only ...,"cellular organisms, Eukaryota, Opisthokonta, M...",Q56P42,NaN,111.29,NaN,NaN,20.000
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,186,0,0.0044,0,False,NaN,NaN,NaN,NaN,...,22438,Homo sapiens (Human),Protein FAM9B,"cellular organisms, Eukaryota, Opisthokonta, M...",Q8IZU0,NaN,120.63,NaN,NaN,20.000
4,MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVK...,1360,0,0.0003,0,False,NaN,NaN,NaN,NaN,...,154943,Homo sapiens (Human),TRAF2 and NCK-interacting protein kinase (EC 2...,"cellular organisms, Eukaryota, Opisthokonta, M...",Q9UKE5,NaN,113.93,NaN,NaN,20.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25112,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...,164,0,0.0000,0,False,NaN,NaN,NaN,NaN,...,18696,Homo sapiens (Human),T-cell surface glycoprotein CD3 zeta chain (T-...,"cellular organisms, Eukaryota, Opisthokonta, M...",P20963,NaN,114.00,NaN,NaN,20.000
25113,MKVNRETKRLYVGGLSQDISEADLQNQFSRFGEVSDVEIITRKDDQ...,1167,0,0.0000,0,False,NaN,NaN,NaN,NaN,...,131616,Homo sapiens (Human),Nucleolar protein 8 (Nucleolar protein Nop132),"cellular organisms, Eukaryota, Opisthokonta, M...",Q76FK4,NaN,112.78,NaN,NaN,20.000
25114,MAGVGDAAAPGEGGGGGVDGPQRDGRGEAEQPGGSGGQGPPPAPQL...,567,0,0.0000,0,False,NaN,NaN,NaN,NaN,...,64260,Homo sapiens (Human),Transmembrane anterior posterior transformatio...,"cellular organisms, Eukaryota, Opisthokonta, M...",Q6NXT6,NaN,113.33,NaN,NaN,20.000
25115,MGASVSRGRAARVPAPEPEPEEALDLSQLPPELLLVVLSHVPPRTL...,283,0,0.0000,0,False,NaN,NaN,NaN,NaN,...,31623,Homo sapiens (Human),F-box only protein 27 (F-box/G-domain protein 5),"cellular organisms, Eukaryota, Opisthokonta, M...",Q8NI29,NaN,111.74,NaN,NaN,20.000


In [57]:
df_full.columns

Index(['Sequence', 'Length', 'virus', 'model_pred_proba_vir', 'model_pred_vir',
       'mistake', 'Baltimore', 'Family', 'Genome Composition', 'Genus',
       'Keywords', 'Mass', 'Organism', 'Protein names', 'Taxonomic lineage',
       'UR50_Cluster ID', 'Virus hosts', 'av_mw', 'genus_human_host',
       'human_host', 'ipw_mistake_balt_vir_host'],
      dtype='object')

In [50]:

# Proceed with training a new model using these weights to adjust for the confounding effect
# Define the RandomForest model with sample weights for IPW adjustment
model_with_ipw = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model using the calculated weights for the training data
model_with_ipw.fit(X_train_transformed, y_train, sample_weight=weights_simplified)

# Since feature names and importances extraction were based on the pipeline, and we're now bypassing the pipeline
# for model retraining with weights, we directly access feature importances from the model
feature_importances_with_ipw = model_with_ipw.feature_importances_

# Create a DataFrame for feature importances with IPW adjustment
importances_df_with_ipw = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances_with_ipw
}).sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Display the top 20 most important features after IPW adjustment
importances_df_with_ipw.head(20)


NameError: name 'X_train_transformed' is not defined